<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>


# <a name="0">Bedrock Agents-Based AI AppBuilder Assistant with Boto3 SDK </a>

## Notebook Overview

In this notebook, we build an agent to act as an Application Builder Assistant. Typically to build a three-tier software application, we need a user interface (UI component), middle tier (aka backend) for business APIs, and a database tier. With this assistant, we can generate SQL queries using a database schema (DDL: Data definition language) and execute them against a database instance, recommend software design best practices using the AWS Well-Architected Framework, and generate code snippets in many of the standard programming languages to improve developer productivity and facilitate rapid development of use-cases.

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This notebook auto-cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#10"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>


##### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.


## LLM Used
- Anthropic Claude 3.0 Sonnet 
- Titan Text Embeddings v2

## Regions tested 
- us-east-1
- us-west-2
- All other regions where agents and respective LLMs are supported are candidates to test this notebook.

## Use-Case Overview
Lets look at when this Application Builder Assistant acts as a coding assistant vs recommends AWS Design Best Practices vs aids in SQL code generation and execution.

First it is important to understand the type of task the user is asking for:

Does the user want to write and validate SQL Query against a database?
- If yes, use the existing DDL schemas to come up with the SQL query.

Does the user want to author some code for say, helper functions like validate email?
- If yes, use prompt engineering techniques to generate the email validation code.

Does the user want recommendation on design best practices?
- If yes, look up with AWS Well-Architected Knowledge Base.

<center><img src="images/agents.drawio.png" alt="architecture diagram for this notebook to demonstrate the conditional workflow for llms. This shows 3 workflows possible via this Application Builder Assistant. 1) Text to SQL - generate SQL statements via natural language and execute it against a local DB 2) web scraped knowledge base on AWS well architected framework - user can ask questions on it 3) Write and explain code via Claude LLM. User can ask any of these three types of questions making it an application builder assistant."  height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>



It demonstrates how to:
1. Select the underlying foundation model (FM) for your agent 
2. Provide a clear and concise agent instruction 
3. Create and associate an action group with an API schema and a Lambda function 
4  Create and associate multiple knowledge bases (2 KBs in this notebook) with the agent
5. Create, invoke, test, and deploy the agent
6. Generate UI/Backend code with LLMs
7. Recommend AWS best practices for system design with the AWS Well-Architected Framework guidelines
8. Generate and execute/validate the SQL from natural language understanding via LLMs, few shot examples, and a database schema as a knowledge base
9. Clean up resources (Be Frugal)


We are using the Retrieval Augmented Generation (RAG) technique with Amazon Bedrock. A RAG implementation consists of two parts:

    A data pipeline that ingests that from documents (typically stored in Amazon S3) into a knowledge base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

    An application that receives a question from the user, looks up the knowledge base for relevant pieces of information (context) and then creates a prompt that includes the question and the context and provides it to an LLM for generating a response.

The data pipeline represents an undifferentiated heavy lifting and can be implemented using Amazon Bedrock Agents for Knowledge Base. We can now connect an S3 bucket to a vector database such as AOSS and have a Bedrock Agent read the objects (html, pdf, text etc.), chunk them, and then convert these chunks into embeddings using Amazon Titan Embeddings model(titan-embed-text-v2:0) and then store these embeddings in AOSS. All of this without having to build, deploy, and manage the data pipeline.



Once the data is available in the Bedrock knowledge base then user questions can be answered using the following system design:


This notebook has the following sections:

1. <a href="#1">Environment configuration</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Setup prefix variables for various agent resources</a>
4. <a href="#4">Creating an agent</a>
5. <a href="#5">Deploy agent and create agent alias</a>
6. <a href="#6">Invoke agent </a>
7. <a href="#7">Code generation examples</a>
8. <a href="#8">System design Q&A with the AWS Well-Architected Framework</a>
9. <a href="#9">SQL generation and execution against a database</a>
10. <a href="#10">Clean up resources</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---


Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.

In [1]:
%%html

<a class="github-button" href="https://github.com/aws-samples/application-builder-assistant-using-bedrock-agents-and-multiple-knowledge-bases" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star AI App Builder Assistant with Agents on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>


## <a name="1">Environment setup and configuration</a>
(<a href="#0">Go to top</a>)

Before starting, let's import the required packages and configure the support variables:

In [ ]:
%%capture
!pip3 install setuptools==58.2.0

In [ ]:
#%%capture
!pip3 install -r requirements.txt --quiet

In [ ]:
%%capture
!pip3 install -U langchain-community
!pip3 install -U opensearch-py
!pip3 install -U requests_aws4auth
!pip3 install -U boto3
!pip3 install -U botocore
!pip3 install -U awscli

In [ ]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
import os, shutil
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from fetch_aws_best_practices import *

from mlu_utils.agents_utils import *
from mlu_utils.agents_infra_utils_with_two_kbs import *
from mlu_utils.show_trace_widget import *
from mlu_utils.summarize_agent_trace import *
from IPython.display import Markdown

In [ ]:
# cleanup trace files to avoid issues
trace_file_path = "./trace_files/"
if os.path.isdir(trace_file_path):
    shutil.rmtree(trace_file_path)
os.mkdir('trace_files')

In [ ]:
clean_up_trace_files("./trace_files/")

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

pp = pprint.PrettyPrinter(width=41, compact=True)

In [ ]:
import ipywidgets as widgets
from IPython.display import JSON
out = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

out_3_tabs_1 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_2 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_3 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_4 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

out_3_turn_1_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_2_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_3_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_4_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))


### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).
```
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

```

</br>

In [ ]:
#test if bedrock model access has been enabled 
input_prompt = "Who was the first person to land on the sun?"
test_llm_call(input_prompt)

In [ ]:
# Check if knowledge base files have been downloaded as per instructions in both of these locations in this project
# 1. `kb_appbuilder/aws_best_practices_2/readMe.txt`
# 2. `kb_appbuilder/northwind_db/readMe.txt`

kb_file_check = check_for_kb_files()

In [ ]:
# throw error and block execution of the rest of the notebook if the KB files do not exist
if not kb_file_check:
    print("Please check for knowledge base files and use the readMe inside the folders kb_appbuilder >> aws_best_practices_2 and kb_appbuilder >> northwind_db folders ")
    raise Exception('Please check for knowledge base files and use the readMe inside the folders kb_appbuilder >> aws_best_practices_2 and kb_appbuilder >> northwind_db folders')
else:
    print("Knowledge base files exist , ready to start setup agent infrastructure")

In [ ]:
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')

session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

### <a name="3">3. Setup agent infrastructure</a>
(<a href="#0">Go to top</a>)

This is the same set of instructions for infrastructure setup as provided in Lab 2a and includes how to:
- Setup for prefix variables with various agent resources
- Create Lambda function for action group
- Create knowledge base 1 for SQL generation with Northwind Database
- Create knowledge base 2 for system design recommendation with AWS Well-Architected Framework
- Create an agent



In [ ]:
%%time
# this takes roughly 10 mins on t3.medium instance.

infra_response = setup_agent_infrastructure()

In [ ]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
knowledge_base_db_id = infra_response["knowledge_base_db_id"]
knowledge_base_aws_id = infra_response["knowledge_base_aws_id"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]
kb_db_collection_name = infra_response["kb_db_collection_name"]
kb_db_bedrock_policy = infra_response["kb_db_bedrock_policy"]
kb_db_aoss_policy = infra_response["kb_db_aoss_policy"]
kb_db_s3_policy = infra_response["kb_db_s3_policy"]
agent_kb_schema_policy = infra_response["agent_kb_schema_policy"]
kb_db_role_name = infra_response["kb_db_role_name"]
kb_db_opensearch_collection_response = infra_response["kb_db_opensearch_collection_response"]
kb_aws_collection_name = infra_response["kb_aws_collection_name"]
kb_aws_bedrock_policy = infra_response["kb_aws_bedrock_policy"]
kb_aws_aoss_policy = infra_response["kb_aws_aoss_policy"]
kb_aws_s3_policy = infra_response["kb_aws_s3_policy"]
kb_aws_role_name = infra_response["kb_aws_role_name"]
kb_aws_opensearch_collection_response = infra_response["kb_aws_opensearch_collection_response"]


### <a name="3">Creating an agent </a>
(<a href="#0">Go to top</a>)

Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model, and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs:

In [ ]:
# Create Agent
agent_instruction = """
Hello, I am AI Application Builder Assistant. 

I am capable of answering the following three categories of questions:
- Best practices for design of software applications using the content inside the AWS best practices and AWS well-architected framework Knowledge Base. I help customers understand AWS best practices for building applications with AWS services.  
- Generate a valid SQLite query for the customer using the database schema inside the Northwind DB knowledge base and then execute the query that answers the question based on the [Northwind] dataset.If the Northwind DB Knowledge Base search function result did not contain enough information to construct a full query try to construct a query to the best of your ability based on the Northwind database schema.
- Generate and Explain code for the customer following standard programming language syntax 

Feel free to ask any questions along those lines!

"""
#foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
# "anthropic.claude-v2"

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="AI Application Builder Assistant",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id:

Let's now store the agent id in a local variable to use it on the next steps

In [ ]:
agent_id = response['agent']['agentId']
agent_id

### Create agent action group
We will now create an agent action group that uses the Lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [ ]:
agent_functions = [
        {
            'name': 'run_query',
            'description': 'this API can only execute queries against Northwind database',
            'parameters': {
                "Query": {
                    "description": "generated SQL Query",
                    "required": True,
                    "type": "string"
                }
            }
        },
        {
            'name': 'query_well_arch_framework',
            'description': 'this API is just a LLM response decorator.',
            'parameters': {
                "query": {
                    "description": "Customer query",
                    "required": True,
                    "type": "string"
                }
            }
        },
        {
            'name': 'gen_code',
            'description': 'this API is just a LLM response decorator.',
            'parameters': {
                "query": {
                    "description": "Customer query",
                    "required": True,
                    "type": "string"
                }
            }
        }
    ]

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='AppBuilderAssistantActionGroup',
    functionSchema={
            'functions': agent_functions
        },
    description='APIs to execute valid SQLite query and help customers with system design by querying the AWS Well-Architected Framework and writing code.'
)

### Allowing agent to invoke action group Lambda
Before using our action group, we need to allow our agent to invoke the Lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the Lambda function created:

In [ ]:
# Create allow invoke permission on Lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Associating the agent to knowledge base 1


In [ ]:
agent_kb_db_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {knowledge_base_db_id} Knowledge Base to generate a valid SQLite Query to answer the questions based on the Northwind database',
    knowledgeBaseId=knowledge_base_db_id 
)

In [ ]:
agent_kb_db_description

### Associating the agent to knowledge base 2

In [ ]:
agent_kb_aws_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {knowledge_base_aws_id} Knowledge Base to answer questions based on the AWS design best practices for enterprise software and AWS well architected framework',
    knowledgeBaseId=knowledge_base_aws_id 
)

### Preparing the agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [ ]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

### <a name="8">Create agent alias to deploy agent</a>
(<a href="#0">Go to top</a>)

We will now create an alias of the agent that can be used to deploy the agent.

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [ ]:
agent_alias

### <a name="9">Invoke agent</a>
(<a href="#0">Go to top</a>)

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [ ]:
USER_PROMPT_TEMPLATE = """Question: {question}

Given an input question, you will use the existing Knowledge Bases on AWS Well-Architected Framework and Northwind DB Knowledge Base. 

- For building and designing software applications, you will use the existing Knowledge Base on AWS well-architected framework to generate a response of the most relevant design principles and links to any documents. This Knowledge Base response can then be passed to the functions available to answer the user question. The final response to the direct answer to the user question. It has to be in markdown format highlighting any text of interest. Remove any backticks in the final response.

- To generate code for a given user question,  you can use the default Large Language model to come up with the response. This response can be in code markdown format. You can optionally provide an explanation for the code.

- To explain code for a given user question, you can use the default Large Language model to come up with the response. 

- For SQL query generation you will ONLY use the existing database schemas in the Northwind DB Knowledge Base to create a syntactically correct SQLite query and then you will EXECUTE the SQL Query using the functions and API provided to answer the question. 
Make sure to use only existing columns and tables exactly as mentioned in database schema. The table names can have spaces in between them. Make sure to wrap table names with square brackets. Make sure to add a semicolon after the end of the SQL statement generated.
Remove any backticks and any html tags like <table><th><tr> in the final response.

Here are a few examples of questions I can help answer by generating and then executing a SQLite query:

- What are the total sales amounts by year?

- What are the top 5 most expensive products? 

- What is the total revenue for each employee?



"""

### <a name="10">Code generation examples </a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to generate code snippets for UI and back-end/middle tier. Change the question to generate the code in different programming languages like Python, JavaScript, Java and others.

<center><img src="images/agents-codecall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on code generation and explanation piece of the AI assistant." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [ ]:
question = "Write a python function to validate email address syntax."

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from mlu_utils.agents_utils import *

In [ ]:
%%time
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 1)

In [ ]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="1", gen_sql=False)

In [ ]:
# Deep dive into agent-workflow steps in each of the tab
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 1)
display(out_3_tabs_1)

In [ ]:
# Wait for tab output async processes to complete
time.sleep(10)

In [ ]:
%%time
# Using Claude-v3 sonnet to generate a summary of the agent's workflow for this conversation turn
turn_1_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="1")
with out_3_turn_1_summary:
    out_3_turn_1_summary.clear_output()
    display(Markdown(turn_1_summary))

In [ ]:

## Display the summary of the agent workflow trace
display(out_3_turn_1_summary)

In [ ]:
code_to_explain = None
with open("mlu_utils/summarize_agent_trace.py", "r") as code_fp:
    code_to_explain = code_fp.read()

with out:
    out.clear_output()
    # display first 500 characters of the py file 
    display(Markdown(code_to_explain[:450]))

display(out)

In [ ]:
question = f"Explain the following code in lucid, natural language to me. \n code to explain : \n {code_to_explain}"

In [ ]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']



session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 2)



In [ ]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="2", gen_sql=False)

In [ ]:
# Deep dive into agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 2)
display(out_3_tabs_2)

In [ ]:
# Wait for tab output async processes to complete
time.sleep(10)

In [ ]:
%%time
# Using Claude-v3 Sonnet to generate a summary of the agent's workflow for this conversation turn
turn_2_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="2")
with out_3_turn_2_summary:
    out_3_turn_2_summary.clear_output()
    display(Markdown(turn_2_summary))

In [ ]:
## Display the summary of the Agent workflow trace
display(out_3_turn_2_summary)


### <a name="11">System design Q&A with AWS Well-Architected Framework</a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to recommend system design setup and best practices based on the robust AWS Well-Architected Framework. 

<center><img src="images/agents-awscall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on the AWS Well-Architected Framework for knowledge base inquiry." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [ ]:
question = "How can I design secure VPCs?"

In [ ]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']


session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 3)


In [ ]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="3", gen_sql=False)

In [ ]:
# Deep dive into Agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 3)
display(out_3_tabs_3)

In [ ]:
# Wait for tab output async processes to complete
time.sleep(10)

In [ ]:
%%time
# Using Claude-v3 Sonnet to generate a summary of the agent's workflow for this conversation turn
turn_3_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="3")
with out_3_turn_3_summary:
    out_3_turn_3_summary.clear_output()
    display(Markdown(turn_3_summary))

In [ ]:
## Display the summary of the agent workflow trace
display(out_3_turn_3_summary)

### <a name="12">SQL generation and execution against a database</a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to generate SQL code based on a database schema of our choice and execute it against a database to validate the accuracy of the generated code.

<center><img src="images/agents-SQLcall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on the Text to SQL generation from natural language questions." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [ ]:
question = "What are the total sales amounts by year?"

In [ ]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']


## Create a random id for session initiator id
session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question), 
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 4)



In [ ]:
# And here is the response if you just want to see agent's reply
%load_ext autoreload
%autoreload 2
from mlu_utils.agents_utils import *
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="4", gen_sql=True)

In [ ]:
# Deep dive into Agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 4)
display(out_3_tabs_4)

In [ ]:
# Wait for tab output async processes to complete
time.sleep(10)

In [ ]:
%%time
# Using Claude-v3 Sonnet to generate a summary of the agent's workflow for this conversation turn
turn_4_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="4")
with out_3_turn_4_summary:
    out_3_turn_4_summary.clear_output()
    display(Markdown(turn_4_summary))

In [ ]:
## Display the summary of the agent workflow trace
display(out_3_turn_4_summary)

### <a name="9">[Be Frugal] Clean up resources </a>
(<a href="#0">Go to top</a>)


##### In the following cell, we offer an option raise an exception to avoid auto-executing the next block of lines and optionally clean up all resources. This is useful when the `Kernel > run all` option is used.

`Please be frugal if you choose to enable this exception in the code cell below. By default it is disabled and all resources will be cleaned up immediately to avoid additional costs.`

##### Within the same kernel session, this will allow experimentation with different prompts without having to recreate agent resources (takes ~5 minutes)

In [ ]:
# This avoids auto-cleanup
raise Exception('Avoiding Auto-Cleanup of Bedrock Agent Resources')

In [ ]:
cleanup_infrastructure(agent_action_group_response, lambda_name, lambda_function, lambda_role_name, agent_id, agent_alias_id, agent_role_name, bucket_name, schema_key, agent_bedrock_policy, agent_s3_schema_policy, agent_kb_schema_policy, kb_db_bedrock_policy, kb_db_aoss_policy, kb_db_s3_policy, kb_db_role_name, kb_db_collection_name, kb_db_opensearch_collection_response, knowledge_base_db_id, kb_aws_bedrock_policy, kb_aws_aoss_policy, kb_aws_s3_policy, kb_aws_role_name, kb_aws_collection_name, kb_aws_opensearch_collection_response, knowledge_base_aws_id)

In [ ]:
## Cleanup S3 separately - it has 2 KBs with GBs of data
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()

s3_client.delete_bucket(Bucket=bucket_name)
print(f"\n bucket DELETED :: {bucket_name}")


## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You